# Sampling

A sampler provides an unlimited series of samples from a joint distribution over one or more random variables. Specifically, a sampler implements the abstract base class, `Sampler` in the `ck.sampling` package.

All samplers have an `rvs` and `condition` property.

The `rvs` property records what random variables are being sampled. They will all be from the same PGM. The order of the random variables provided by `rvs` is important.

The `condition` property records the conditions placed on the sampler at construction time.

A Sampler will either iterate over `Instance` objects (each instance having states co-indexed
with its `rvs`) or will iterate over integers (each integer being a single state index). Whether a Sampler iterates
over `Instance` objects or integers is determined by the implementation.

If a sampler iterates over integers, then the length of its `rvs` is 1.

To explain samplers, we will use an example PGM...

In [1]:
from ck.example import Asia
from ck.pgm import RVMap

pgm = Asia()  # An example PGM to demonstrate samplers

rvs = RVMap(pgm)  # Provide easy access to the PGM random variables

## Forward Sampling

CK provides an implementation of the Forward Sampling algorithm, `ForwardSampler` in the `ck.sampling.forward_sampler` package. A forward sampler can be constructed directly from a PGM, so long as the PGM represents a Bayesian network.

In [2]:
from ck.sampling.forward_sampler import ForwardSampler

forward_sampler = ForwardSampler(pgm)

A sampler provides an infinite iterator over instances.

In this example we limit ourselves to 8 samples.

In [3]:
print(*forward_sampler.rvs)  # Show the random variables

# Draw some samples
for i, inst in enumerate(forward_sampler, start=1):
    print(i, inst)
    if i == 8:
        break

asia tub smoke lung bronc either xray dysp
1 [1, 1, 0, 1, 0, 1, 1, 0]
2 [1, 1, 0, 1, 0, 1, 1, 0]
3 [1, 1, 1, 1, 1, 1, 1, 1]
4 [1, 1, 1, 1, 1, 1, 1, 1]
5 [1, 1, 1, 1, 1, 1, 1, 1]
6 [1, 1, 0, 1, 1, 1, 1, 1]
7 [1, 1, 0, 1, 0, 1, 1, 1]
8 [1, 1, 1, 1, 1, 1, 1, 0]


A simple way to take a limited number of samples is to use `take`.

In [4]:
for inst in forward_sampler.take(8):
    print(inst)

[1, 1, 1, 1, 1, 1, 1, 1]
[1, 1, 1, 1, 1, 1, 1, 1]
[1, 1, 1, 1, 0, 1, 1, 0]
[1, 1, 1, 1, 1, 1, 1, 0]
[1, 1, 0, 1, 1, 1, 1, 1]
[1, 1, 1, 1, 0, 1, 1, 1]
[1, 1, 1, 1, 1, 1, 0, 1]
[1, 1, 1, 1, 1, 1, 1, 1]


Most samplers can also be constructed to sample a PGM with some random variables conditioned.

In [5]:
# Construct a condition where smoke = no, lung = yes, xray = no.
condition = (rvs.smoke('no'), rvs.lung('yes'), rvs.xray('no'))

forward_sampler = ForwardSampler(pgm, condition=condition)

for inst in forward_sampler.take(8):
    print(inst)

[1, 1, 1, 0, 0, 0, 1, 0]
[1, 1, 1, 0, 1, 0, 1, 1]
[1, 1, 1, 0, 0, 0, 1, 0]
[1, 1, 1, 0, 1, 0, 1, 0]
[1, 1, 1, 0, 0, 0, 1, 0]
[1, 1, 1, 0, 1, 0, 1, 0]


[1, 1, 1, 0, 1, 0, 1, 0]


[1, 1, 1, 0, 1, 0, 1, 0]


A forward sample is very efficient for sampling Bayesian networks, without conditioning. However, conditioning can make a forward sampler slow, especially for low-probability conditions, as the forward sample needs to use rejection sampling to implement the condition.

The timing difference may be apparent in the two examples above.

## Uniform Sampler

Another sampler that can be created directly from a PGM is a `UniformSampler` in package `ck.sampling.uniform_sampler`. It only requires a list of random variables as it will sample them assuming a uniform distribution.

A `UniformSampler` is very computationally efficient (as it does not require any analysis of the probability distribution of a PGM). The main purpose of this sampler is to provide a trivial baseline (in terms of speed and accuracy) when comparing sampling algorithms.

In [6]:
from ck.sampling.uniform_sampler import UniformSampler

uniform_sampler = UniformSampler(pgm.rvs)

print(*uniform_sampler.rvs)

for inst in uniform_sampler.take(8):
    print(inst)

asia tub smoke lung bronc either xray dysp
[1, 1, 0, 0, 1, 0, 0, 1]
[1, 0, 1, 1, 0, 1, 1, 0]
[0, 1, 0, 1, 0, 1, 1, 1]
[1, 1, 1, 0, 1, 0, 0, 0]
[1, 1, 0, 0, 1, 1, 0, 1]
[0, 1, 1, 1, 0, 1, 1, 0]
[0, 0, 0, 0, 1, 0, 1, 1]
[0, 0, 1, 0, 1, 0, 0, 1]


## Gibbs Sampler

Gibbs sampling is a common approach to sampling a probability distribution. There are many explanations available for this approach (see https://en.wikipedia.org/wiki/Gibbs_sampling).

Gibbs sampling is a type of "dependant" sampling, which is where a Markov chain is used to generate samples - nearby samples in the sequence are correlated. It may not be suitable if independent samples are desired.

To perform Gibbs sampling in CK, a PGM must first be compiled to a `WMCProgram`. Here is an example.

In [7]:
from ck.pgm_compiler import DEFAULT_PGM_COMPILER
from ck.pgm_circuit.wmc_program import WMCProgram

wmc = WMCProgram(DEFAULT_PGM_COMPILER(pgm))

gibbs_sampler = wmc.sample_gibbs()

for inst in gibbs_sampler.take(8):
    print(inst)

[0, 0, 1, 1, 1, 0, 0, 1]
[0, 0, 1, 1, 1, 0, 0, 0]
[1, 0, 1, 1, 0, 0, 0, 0]
[1, 0, 0, 1, 1, 0, 0, 0]
[1, 0, 1, 1, 1, 0, 0, 1]
[1, 0, 0, 1, 0, 0, 0, 0]
[1, 0, 0, 1, 1, 0, 0, 1]
[1, 0, 0, 1, 1, 0, 0, 1]


## Metropolis–Hastings Sampler

Metropolis–Hastings sampling is another common approach to sampling a probability distribution (see https://en.wikipedia.org/wiki/Metropolis%E2%80%93Hastings_algorithm).

Metropolis–Hastings sampling is also a type of "dependant" sampling - nearby samples in the sequence are correlated.

CK creates a Metropolis–Hastings sampler from a `WMCProgram`. Here is an example using the `WMCProgram` created above.

In [8]:
metropolis_sampler = wmc.sample_metropolis()

for inst in metropolis_sampler.take(8):
    print(inst)

[1, 1, 0, 0, 1, 0, 0, 0]
[1, 1, 0, 0, 0, 0, 0, 0]
[1, 1, 0, 0, 0, 0, 0, 0]
[1, 1, 0, 0, 0, 0, 0, 0]
[1, 1, 0, 0, 0, 0, 0, 0]
[1, 1, 0, 0, 1, 0, 0, 0]
[1, 1, 0, 0, 1, 0, 0, 0]
[1, 1, 0, 0, 0, 0, 0, 0]


## Rejection Sampler

Rejection sampling is a technique for converting samples from a source distribution into a samples from a target distribution (see https://en.wikipedia.org/wiki/Rejection_sampling). It does this by randomly discarding samples from the source distribution to match the target distribution.

In CK the source distribution is from an independent, uniform sampler. In this case, rejection sampling is a type of "independent sampling" - nearby samples in the sequence are _not_ correlated.

CK creates a rejection sampler from a `WMCProgram`. Here is an example using the `WMCProgram` created above.




In [9]:
rejection_sampler = wmc.sample_rejection()

for inst in rejection_sampler.take(8):
    print(inst)

[1, 1, 0, 1, 0, 1, 1, 0]
[1, 1, 1, 1, 1, 1, 1, 1]
[1, 1, 0, 1, 0, 1, 1, 0]
[1, 1, 0, 1, 0, 1, 1, 0]
[1, 1, 0, 1, 0, 1, 1, 0]
[1, 1, 1, 1, 0, 1, 1, 0]
[1, 1, 1, 1, 1, 1, 1, 1]
[1, 1, 0, 1, 0, 1, 1, 0]


## WMC Direct Sampler

CK provides two custom sampling methods that exploit efficiencies gained from compiling PGMs and circuits. They are called "WMC Direct" sampler and "Marginals Direct" sampler. They are both based on "inverse transform sampling" which is a type of "independent" sampling - nearby samples in the sequence are _not_ correlated (see https://en.wikipedia.org/wiki/Inverse_transform_sampling).

The algorithms are described and evaluated in the publication: Suresh, S., Drake, B. (2025). Sampling of Large Probabilistic Graphical Models Using Arithmetic Circuits. AI 2024: Advances in Artificial Intelligence. AI 2024. Lecture Notes in Computer Science, vol 15443. https://doi.org/10.1007/978-981-96-0351-0_13.

So long as a PGM can be efficiently compiled to an arithmetic circuit, then a WMC Direct sampler is a computationally efficient independent sampler, even for complex probability distributions and when the probability distribution is conditioned after compilation.

Here is an example of the WMC Direct sampler using the `WMCProgram` created above.

In [10]:
wmc_direct_sampler = wmc.sample_direct()

for inst in wmc_direct_sampler.take(8):
    print(inst)

[1, 1, 1, 1, 1, 1, 1, 0]
[1, 1, 1, 1, 0, 1, 1, 0]
[1, 1, 1, 1, 0, 1, 1, 0]
[1, 1, 1, 1, 0, 1, 1, 0]
[1, 1, 1, 1, 1, 1, 1, 1]
[1, 1, 1, 1, 1, 1, 1, 1]
[1, 1, 0, 1, 0, 1, 1, 0]
[1, 1, 0, 1, 1, 1, 1, 1]


## Marginals Direct Sampler

A variation on the "WMC Direct" sampler is the "Marginals Direct" sampler.

The Marginals Direct sampler is based on "inverse transform sampling" but calculates an inverse cumulative probability function differently to WMC Direct. Specifically, is uses the so-called differential approach to computing marginal distributions. This approach to marginal distributions is described in the publication: Adnan Darwiche (2003). A differential approach to inference in Bayesian networks. J. ACM 50, 3 (May 2003), 280–305. https://doi.org/10.1145/765568.765570.

The Marginals Direct algorithm is described and evaluated in the publication: Suresh, S., Drake, B. (2025). Sampling of Large Probabilistic Graphical Models Using Arithmetic Circuits. AI 2024: Advances in Artificial Intelligence. AI 2024. Lecture Notes in Computer Science, vol 15443. https://doi.org/10.1007/978-981-96-0351-0_13.

So long as a PGM can be efficiently compiled to an arithmetic circuit, then a Marginals Direct sampler is a computationally efficient independent sampler, even for complex probability distributions and when the probability distribution is conditioned after compilation.

Here is an example of the Marginals Direct sampler using the `WMCProgram` created above.



In [11]:
from ck.pgm_circuit.marginals_program import MarginalsProgram

marginals = MarginalsProgram(DEFAULT_PGM_COMPILER(pgm))

marginals_direct = marginals.sample_direct()

for inst in marginals_direct.take(8):
    print(inst)

[1, 1, 0, 1, 0, 1, 1, 0]
[1, 1, 0, 1, 0, 1, 1, 0]
[1, 1, 1, 1, 0, 1, 1, 1]
[1, 1, 1, 1, 1, 1, 1, 1]
[1, 1, 1, 1, 1, 1, 1, 1]
[1, 1, 0, 1, 1, 1, 1, 0]
[1, 1, 1, 1, 1, 1, 1, 1]
[1, 1, 0, 0, 1, 0, 0, 0]


## Options When Creating Samplers

When creating a Direct sampler (from `WMCProgram` or `MarginaslProgram`) several options are available to customize the sampler. Here is the function signature.

```
sample_direct(
        rvs: Optional[RandomVariable | Sequence[RandomVariable]] = None,
        *,
        condition: Condition = (),
        rand: Random = random,
        chain_pairs: Sequence[Tuple[RandomVariable, RandomVariable]] = (),
        initial_chain_condition: Condition = (),
) -> Sampler
```

`rvs` is a list of random variables to sample. They should all be from the source PGM.

If `rvs` is a random variable object, then samples are yielded as integers representing state indexes from the random variable.

If `rvs` is an array of random variables, then samples are yielded as `Instance` objects. Each instance is a list of integers co-indexed with the given random variables, and where each integer represents a state index from the corresponding random variable.

If `rvs` is None, then all random variables from the source PGM will be used, in the order reported by the PGM.

`condition` is a collection of zero or more conditioning indicators. Semantically, the indicators of the given condition are grouped by random variable. The condition is interpreted as the conjunction of the groups, and the disjunction of the states within each group. For example, given condition for the Student PGM,
```
    grade('1'), intelligent('Yes'), grade('3')
```
then the condition interpretation is: (grade = 1 or grade = 3) and intelligent = Yes.

Note that `WMCProgram` and `MarginaslProgram` objects are created from a `PGMCircuit`, which may already have some condition compiled into the circuit. If both the circuit has a condition and the sampler has a condition, they are treated conjunctively, even if the same random variable is referenced in the conditions. For example, given
```
    circuit condition: grade('2')
    sampler condition: grade('1'), intelligent('Yes'), grade('3')
```
then the condition interpretation is: grade = 2 and (grade = 1 or grade = 3) and intelligent = Yes. Note that in this example, the two conditions form a contradiction so all possible worlds have zero probability and no sampler could function normally.

`rand` is an optional stream of random numbers, conforming to the `Random` protocol (package `ck.random_extras`). The default is the standard Python `random` package`.

The parameters `chain_pairs` and `initial_chain_condition` provide a mechanism to construct complex Markov chains from any PGM.

`chain_pairs` is a collection of pairs of random variables, each random variable
 must one that is actually being sampled. Given a pair (_from_rv_, _to_rv_) the state of _from_rv_ is used
 as a condition for _to_rv_ prior to generating the next sample.
 Caution is advised to ensure that such chains of conditions cannot contradict with conditions provided to the sampler or conditions compiled into the circuit. If a state is reached where there is a contradiction, then all possible worlds have zero probability and no sampler could function normally.

`initial_chain_condition` are condition indicators (with the same format as `condition`)
 for the initialisation of the _to_rv_ random variables mentioned in `chain_pairs`.
 Caution is advised to ensure that given initial conditions do not contradict with  conditions provided to the sampler or conditions compiled into the circuit. If there is a contradiction, all possible worlds have zero probability and no sampler could function normally.

Dependant samplers (Gibbs and Metropolis-Hastings) will have options specific to that style of sampling: `skip`, `burn_in` and `pr_restart`.

`skip` is an integer ≥ 0 specifying how may samples to discard before a sample is provided. Values > 0 can be used to help de-correlate nearby samples.

`burn_in` determines how many iterations to perform at the start, before providing the first sample. This is provided to deal with need to "warm-up" a Markov chain.

`pr_restart`: the chance of re-initialising each time a sample is provided. If restarted then the sampler state is randomised and burn-in is performed again. This is provided to deal with the problem where a Markov chain gets stuck in a mode of high probability and fails to explore other important areas of the sample space.

In general, creating a sampler can be customized by optional parameters similar to those above. Which options are available depends on the sampler implementation details.
